In [15]:
%matplotlib inline

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from common import create_engine, display_all

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters() # converters e.g. for datetime in plots

In [3]:
engine = create_engine('db-conf.json', 'dp.kamko.dev')
pd.sql = lambda sql: pd.read_sql(sql, con=engine)

In [8]:
df = pd.sql('''
SELECT sq.*, s.url as source
FROM (
         SELECT afb.*, row_number() over (partition by url order by sync_date desc) rn
         FROM article_fb_engagement afb) sq
JOIN article a on sq.url = a.url
JOIN source s on a.source_id = s.id
WHERE sq.rn = 1;
''')
df['popularity'] = df['reaction_count'] + df['comment_count'] + df['share_count']
df = df[~df.source.str.endswith(('.sk', '.cz'))]

In [10]:
display_all(df.groupby('source')['popularity'].describe().sort_values('count', ascending=False))

,count,mean,std,min,25%,50%,75%,max
source,,,,,,,,
http://naturalnews.com,20986.0,1224.213762,41273.544730,0.0,19.00,88.0,250.00,5197586.0
http://naturalnewsblogs.com,17756.0,86.888263,2294.673255,0.0,0.00,0.0,1.00,203855.0
http://neurosciencenews.com,13160.0,272.636246,4005.574856,0.0,0.00,0.0,46.25,294411.0
http://statnews.com,12298.0,215.515124,1910.789779,0.0,0.00,0.0,24.00,82773.0
http://hsionline.com,8178.0,0.821228,12.143201,0.0,0.00,0.0,0.00,726.0
http://naturalsociety.com,7350.0,10.235374,598.635593,0.0,0.00,0.0,0.00,49319.0
http://healthnutnews.com,7259.0,1656.149883,23495.200324,0.0,4.00,180.0,743.00,1810067.0
http://healthimpactnews.com,6269.0,354.487159,10888.347959,0.0,0.00,0.0,9.00,841807.0
http://infowars.com,6026.0,52.919847,380.053749,0.0,0.00,0.0,8.00,16902.0


In [12]:
df.popularity.describe().apply(lambda x: '%.3f' % x)

count     168921.000
mean        1081.007
std        23680.102
min            0.000
25%            0.000
50%            1.000
75%           81.000
max      5197586.000
Name: popularity, dtype: object

In [24]:
df.popularity.quantile([i / 100 for i in range(0, 100, 5)])

0.00       0.0
0.05       0.0
0.10       0.0
0.15       0.0
0.20       0.0
0.25       0.0
0.30       0.0
0.35       0.0
0.40       0.0
0.45       0.0
0.50       1.0
0.55       3.0
0.60       7.0
0.65      17.0
0.70      40.0
0.75      81.0
0.80     150.0
0.85     282.0
0.90     602.0
0.95    1722.0
Name: popularity, dtype: float64